In [4]:
#raw text
text='Define which data represents "ham" class and which data represents "spam" class for machine learning algorithm.'
delimiters = ['"',"."]
words = []
current_word = ""
#look at each character in text
for char in text:
    #if space is found
    if char==" ":
        #current_word is not empty
        if not current_word=="":
            words.append(current_word.lower())
            current_word = ""
    #if char is delimiter
    elif char in delimiters:
        if current_word == "":
            words.append(char)
        else:
            words.append(current_word)
            words.append(char)
            current_word = ""
    else:
        current_word += char
#tokenized words from raw text
print(words)
        

['define', 'which', 'data', 'represents', '"', 'ham', '"', 'class', 'and', 'which', 'data', 'represents', '"', 'spam', '"', 'class', 'for', 'machine', 'learning', 'algorithm', '.']


In [9]:
import os
import codecs
#reading data for analysis
def read_in(folder):
    #gets files in directory
    files = os.listdir(folder)
    a_list = []
    for a_file in files:
        #collecting text from each file
        if not a_file.startswith("."):
            f = codecs.open(folder + a_file,"r",encoding = "ISO-8859-1", errors="ignore")
            a_list.append(f.read())
            f.close()
    return a_list
#reading spam text
spam_list = read_in("enron1/spam/")
#reading ham text
ham_list = read_in("enron1/ham/")
print(len(spam_list))
print(len(ham_list))
print(spam_list[0])
print(ham_list[0])

1500
3672
Subject: dobmeos with hgh my energy level has gone up ! stukm
introducing
doctor - formulated
hgh
human growth hormone - also called hgh
is referred to in medical science as the master hormone . it is very plentiful
when we are young , but near the age of twenty - one our bodies begin to produce
less of it . by the time we are forty nearly everyone is deficient in hgh ,
and at eighty our production has normally diminished at least 90 - 95 % .
advantages of hgh :
- increased muscle strength
- loss in body fat
- increased bone density
- lower blood pressure
- quickens wound healing
- reduces cellulite
- improved vision
- wrinkle disappearance
- increased skin thickness texture
- increased energy levels
- improved sleep and emotional stability
- improved memory and mental alertness
- increased sexual potency
- resistance to common illness
- strengthened heart muscle
- controlled cholesterol
- controlled mood swings
- new hair growth and color restore
read
more at this website
un

In [12]:
import random
import nltk
from nltk import word_tokenize #prebuilt tokenizer
nltk.download('punkt')
#data structure for spam detector
all_emails = [(email_content, "spam") for email_content in spam_list]
all_emails += [(email_content, "ham") for email_content in ham_list]
random.seed(42)
random.shuffle(all_emails)# randomizes email list
print(f"Dataset size = {str(len(all_emails))} emails")
def tokenize(input):
    word_list=[]
    for word in word_tokenize(input):
        word_list.append(word)
    return word_list
input = "What's the best way to split a sentence into words?"
print(tokenize(input))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joshu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


Dataset size = 5172 emails
['What', "'s", 'the', 'best', 'way', 'to', 'split', 'a', 'sentence', 'into', 'words', '?']


In [14]:
def get_feature(text):
    features = {} #Collect features as a dictionary of words found
    word_list = [word for word in word_tokenize(text.lower())]
    for word in word_list:
        features[word] = True
    return features
all_features = [(get_feature(email), label) for (email, label) in all_emails]
print(get_feature("Participate In Our New Lottery NOW!"))
print(len(all_features[0][0]))
print(len(all_features[99][0]))


{'participate': True, 'in': True, 'our': True, 'new': True, 'lottery': True, 'now': True, '!': True}
27
55


In [15]:
from nltk import NaiveBayesClassifier, classify

def train(features, proportion):
    train_size = int(len(features)*proportion)
    train_set = features[:train_size]
    test_set = features[train_size:]
    print(f"Training set size ={str(len(train_set))} emails")
    print(f"Test set size = {str(len(test_set))} emails")
    #passing training data into classifier
    classifier = NaiveBayesClassifier.train(train_set)
    return train_set, test_set, classifier

train_set, test_set, classifier = train(all_features, 0.8)
print(train_set)
print(test_set)
    

Training set size =4137 emails
Test set size = 1035 emails


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
def evaluate(train_set,test_set, classifier):
    print(f"Accuracy on the training set = {str(classify.accuracy(classifier,train_set))}")
    print(f"Accuracy on the test set = {str(classify.accuracy(classifier,test_set))}")
    classifier.show_most_informative_features(50)
evaluate(train_set,test_set,classifier)

Accuracy on the training set = 0.9632583998066232
Accuracy on the test set = 0.9468599033816425
Most Informative Features
               forwarded = True              ham : spam   =    198.3 : 1.0
                    2004 = True             spam : ham    =    156.6 : 1.0
                     nom = True              ham : spam   =    126.3 : 1.0
            prescription = True             spam : ham    =    122.9 : 1.0
                    2001 = True              ham : spam   =    107.8 : 1.0
                    pain = True             spam : ham    =     98.8 : 1.0
                     ect = True              ham : spam   =     84.0 : 1.0
                  health = True             spam : ham    =     82.7 : 1.0
                featured = True             spam : ham    =     74.7 : 1.0
              nomination = True              ham : spam   =     73.7 : 1.0
             medications = True             spam : ham    =     69.9 : 1.0
                  differ = True             spam : ha

In [19]:
from nltk.text import Text

def concordance(data_list, search_word):
    for email in data_list:
        word_list = [word for word in word_tokenize(email.lower())]
        text_list = Text(word_list)
        if search_word in word_list:
            text_list.concordance(search_word)
        
print("Stock in HAM:")
concordance(ham_list, "stocks")
print("Stock in SPAM:")
concordance(spam_list, "stocks")

Stock in HAM:
Displaying 1 of 1 matches:
ad my portfolio is diversified into stocks that have lost even more money than
Displaying 1 of 1 matches:
ur member directory . * follow your stocks and news headlines , exchange files
Displaying 1 of 1 matches:
ur member directory . * follow your stocks and news headlines , exchange files
Displaying 1 of 1 matches:
ur member directory . * follow your stocks and news headlines , exchange files
Stock in SPAM:
Displaying 2 of 2 matches:
ims and do your own due diligence . stocks to play ( s 2 p ) profiles are not 
s obtained . investing in micro cap stocks is extremely risky and , investors 
Displaying 1 of 1 matches:
cautions that small and micro - cap stocks are high - risk investments and tha
Displaying 1 of 1 matches:
s obtained . investing in micro cap stocks is extremely risky and , investors 
Displaying 3 of 3 matches:
ancements but may be one of the few stocks left in this industry group that is
his email pertaining to investing , stocks ,

Displaying 3 of 3 matches:
 plays . widespread gains in energy stocks are inflating the portfolios of agg
st levels of the year , with energy stocks outperforming all other market sect
utions that sma | | and micro - cap stocks are high - risk investments and tha
Displaying 3 of 3 matches:
 statements . as with many microcap stocks , today ' s company has additiona |
is report pertaining to investing , stocks , securities must be understood as 
ntative before deciding to trade in stocks featured within this report . none 
Displaying 2 of 2 matches:
his email pertaining to investing , stocks , securities must be understood as 
ntative before deciding to trade in stocks featured within this email . none o
Displaying 1 of 1 matches:
scovering value in natural resource stocks elgin resources ( elr - tsx ) extra
Displaying 2 of 2 matches:
 the last 12 months , many of these stocks made tripie and even quadruple retu
one trade tuesday ! go mogi . penny stocks are considered highly speculativ